<a href="https://colab.research.google.com/github/tanatswajulius/notebook-projects/blob/main/recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

rng = np.random.default_rng(7)

pd.set_option("display.precision", 3)

def hline(title=""):
    print("\n" + "="*80)
    if title:
        print(title)
        print("-"*80)


In [ ]:
  zones = pd.DataFrame({
    "zone_id": range(12),
    "road_quality": rng.integers(1, 5, 12),     # 1 (poor) .. 4 (good)
    "police_presence": rng.integers(1, 5, 12)   # proxy for perceived safety
})

n_users = 2000
users = pd.DataFrame({
    "user_id": range(n_users),
    "home_zone": rng.integers(0, 12, n_users),
    "price_sensitivity": rng.uniform(0, 1, n_users),
    "safety_pref": rng.uniform(0, 1, n_users),
    "typical_hour": rng.integers(6, 22, n_users)
})

def simulate_trips(n=15000, candidates_per_request=3, seed=None):
    if seed is not None:
        local_rng = np.random.default_rng(seed)
    else:
        local_rng = rng

    rows = []
    for _ in range(n):
        u = users.sample(1, random_state=int(local_rng.integers(1e9))).iloc[0]
        t_hour = int(np.clip(local_rng.normal(u.typical_hour, 2), 6, 23))
        surge = float(np.clip(local_rng.normal(1.0 + (t_hour in [7,8,17,18])*0.30, 0.1), 1.0, 2.0))
        rain = local_rng.choice([0,1], p=[0.85,0.15])
        zone = zones.sample(1, random_state=int(local_rng.integers(1e9))).iloc[0]

        for k in range(candidates_per_request):
            walk_m = local_rng.uniform(50, 350)         # meters to walk
            curb_safety = local_rng.uniform(0, 1)       # curb suitability
            eta_min = local_rng.uniform(2, 9) + (0.8 * rain)

            #  comfort score: user + environment interactions
            comfort = (
                0.45*curb_safety
                + 0.20*(4 - zone.road_quality)/4
                + 0.10*(1 if walk_m < 200 else 0)
                + 0.10*(rain == 0)
                + 0.15*(1 - u.safety_pref*0.4) # if safety_pref high, penalize poor curb_safety via road quality term above
            )

            # completion probability (influenced by price sensitivity and ETA)
            p_complete = (
                0.55*comfort
                + 0.25*(1 - u.price_sensitivity*(surge - 1)/1)
                + 0.20*(1 - eta_min/10)
            )
            p_complete = np.clip(p_complete, 0.05, 0.95)
            completed = int(local_rng.random() < p_complete)

            rows.append({
                "request_id": _,
                "user_id": int(u.user_id),
                "hour": t_hour,
                "surge": surge,
                "rain": rain,
                "zone_id": int(zone.zone_id),
                "road_quality": int(zone.road_quality),
                "pickup_k": k,
                "walk_m": walk_m,
                "curb_safety": curb_safety,
                "eta_min": eta_min,
                "completed": completed
            })
    return pd.DataFrame(rows)

trips = simulate_trips()
hline("Sample candidate rows")
display(trips.sample(5, random_state=1))



Sample candidate rows
--------------------------------------------------------------------------------


,request_id,user_id,hour,surge,rain,zone_id,road_quality,pickup_k,walk_m,curb_safety,eta_min,completed
25275,8425,1641,20,1.068,0,3,4,0,99.798,0.736,7.567,1
12561,4187,270,8,1.241,0,10,2,0,283.891,0.387,8.023,0
24707,8235,446,14,1.166,0,3,4,2,221.003,0.967,2.229,1
25066,8355,813,14,1.000,0,5,4,1,80.166,0.408,6.396,0
33429,11143,514,21,1.000,0,2,3,0,104.364,0.051,7.648,0


In [ ]:
FEATURES = ["hour","surge","rain","road_quality","walk_m","curb_safety","eta_min"]
TARGET = "completed"

X = trips[FEATURES]
y = trips[TARGET]

Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

pickup_model = GradientBoostingClassifier(random_state=42)
pickup_model.fit(Xtr, ytr)

auc = roc_auc_score(yte, pickup_model.predict_proba(Xte)[:,1])
hline("Pickup model training")
print(f"AUC: {auc:.3f}")



Pickup model training
--------------------------------------------------------------------------------
AUC: 0.606


In [ ]:
def recommend_pickup(candidates_df, model=pickup_model):
    proba = model.predict_proba(candidates_df[FEATURES])[:,1]
    out = candidates_df.copy()
    out["p_complete_pred"] = proba

    # human-friendly “why” text
    def why(row):
        reasons = []
        if row["curb_safety"] >= candidates_df["curb_safety"].median():
            reasons.append("safer curb")
        # compare to candidate medians for a simple “why”
        if row["eta_min"] <= candidates_df["eta_min"].median():
            reasons.append("faster ETA")
        if row["walk_m"] <= candidates_df["walk_m"].median():
            reasons.append("shorter walk")
        if not reasons:
            reasons.append("balanced tradeoffs")
        return ", ".join(reasons)

    out["why"] = out.apply(why, axis=1)
    return out.sort_values("p_complete_pred", ascending=False)

# tweak hour/surge/rain to see behavior
request_candidates = pd.DataFrame({
    "hour":[18,18,18],
    "surge":[1.25,1.25,1.25],
    "rain":[0,0,0],
    "road_quality":[3,3,3],
    "walk_m":[80,210,320],
    "curb_safety":[0.9,0.6,0.4],
    "eta_min":[3.2,4.8,3.9],
    "pickup_k":[0,1,2],
    # placeholders columns
    "user_id":[-1,-1,-1],
    "zone_id":[7,7,7]
})

hline("Pickup recommendation")
display(recommend_pickup(request_candidates)[["pickup_k","p_complete_pred","walk_m","curb_safety","eta_min","why"]])



Pickup recommendation
--------------------------------------------------------------------------------


,pickup_k,p_complete_pred,walk_m,curb_safety,eta_min,why
0,0,0.775,80,0.9,3.2,"safer curb, faster ETA, shorter walk"
1,1,0.630,210,0.6,4.8,"safer curb, shorter walk"
2,2,0.601,320,0.4,3.9,faster ETA


In [ ]:
#zone-by-hour stats
zone_stats = trips.groupby(["zone_id","hour"]).agg(
    conv_rate=("completed","mean"),
    demand=("completed","size")
).reset_index()

def bandit_choose_zone(current_hour, epsilon=0.10, seed=None):
    local_rng = np.random.default_rng(seed) if seed is not None else rng
    sub = zone_stats[zone_stats["hour"]==current_hour].copy()
    # fallback if no rows (shouldn't happen with our sim)
    if sub.empty:
        return int(zones.sample(1, random_state=int(local_rng.integers(1e9))).zone_id)

    if local_rng.random() < epsilon:
        # explore
        return int(sub.sample(1, random_state=int(local_rng.integers(1e9))).zone_id)
    # exploit
    return int(sub.sort_values("conv_rate", ascending=False).zone_id.iloc[0])

hline("Driver reposition")
for hr in [11, 17]:
    z = bandit_choose_zone(hr)
    row = zone_stats[(zone_stats.hour==hr) & (zone_stats.zone_id==z)].iloc[0]
    print(f"Hour {hr:02d}: go to Zone {z} (conv_rate={row.conv_rate:.3f}, demand={int(row.demand)})")



Driver reposition
--------------------------------------------------------------------------------
Hour 11: go to Zone 7 (conv_rate=0.709, demand=237)
Hour 17: go to Zone 9 (conv_rate=0.692, demand=240)


In [ ]:
from sklearn.utils import shuffle

# subset to simulate promo assignments & observed outcomes
promo = shuffle(trips.sample(6000, random_state=1).copy(), random_state=1)
promo["offered"] = rng.choice([0,1], size=len(promo), p=[0.6,0.4])
promo = promo.merge(users[["user_id","price_sensitivity"]], on="user_id", how="left")

# Observed outcome: base completed + uplift for offered (bigger for high price sensitivity)
base_prob = 0.02 + 0.60*promo["completed"].astype(float)
uplift_true = promo["offered"] * (0.05 + 0.15*promo["price_sensitivity"].fillna(0.5))
obs_prob = np.clip(base_prob + uplift_true, 0, 0.98)
promo["obs_complete"] = (rng.random(len(promo)) < obs_prob).astype(int)

UPLIFT_FEATS = FEATURES + ["price_sensitivity"]

treat = promo[promo.offered==1]
ctrl  = promo[promo.offered==0]

A = GradientBoostingClassifier(random_state=0).fit(treat[UPLIFT_FEATS], treat["obs_complete"])
B = GradientBoostingClassifier(random_state=0).fit(ctrl [UPLIFT_FEATS], ctrl ["obs_complete"])

promo["uplift_pred"] = (
    A.predict_proba(promo[UPLIFT_FEATS])[:,1]
    - B.predict_proba(promo[UPLIFT_FEATS])[:,1]
)

# budget: top 10% uplift
cut = np.quantile(promo["uplift_pred"], 0.90)
recommend_promos = promo[promo["uplift_pred"] >= cut]

hline("Promo targeting summary")
print(f"Budgeted users: {recommend_promos['user_id'].nunique()}")
display(recommend_promos[["user_id","uplift_pred","price_sensitivity","hour","surge"]].head(8))



Promo targeting summary
--------------------------------------------------------------------------------
Budgeted users: 387


,user_id,uplift_pred,price_sensitivity,hour,surge
9,960,0.426,0.886,18,1.349
10,822,0.292,0.778,6,1.000
11,375,0.284,0.022,19,1.003
14,1908,0.330,0.979,18,1.190
16,902,0.310,0.905,16,1.005
40,1714,0.426,0.842,18,1.397
63,285,0.293,0.796,6,1.040
64,1689,0.371,0.720,7,1.455


In [ ]:
def simulate_random_request(n_requests=200):
    rows = []
    for _ in range(n_requests):
        hour = int(rng.integers(6, 23))
        surge = float(np.round(rng.uniform(1.0, 1.6), 2))
        rain = int(rng.choice([0,1], p=[0.9,0.1]))
        road_quality = int(rng.integers(1, 5))
        # make three candidates per request
        for k in range(3):
            rows.append(dict(
                hour=hour, surge=surge, rain=rain, road_quality=road_quality,
                walk_m=float(rng.uniform(50, 350)),
                curb_safety=float(rng.uniform(0,1)),
                eta_min=float(rng.uniform(2, 9) + 0.8*rain),
                pickup_k=k,
                user_id=-1, zone_id=0
            ))
    return pd.DataFrame(rows)

requests = simulate_random_request(300)
# score candidates
requests["p_complete_pred"] = pickup_model.predict_proba(requests[FEATURES])[:,1]

# AI policy: pick argmax per request group of 3
ai_choice = requests.sort_values(["hour","surge","rain"]).groupby(
    np.arange(len(requests))//3
).apply(lambda g: g.loc[g["p_complete_pred"].idxmax()])

# baseline policy: pick closest walk distance per request group of 3
closest_choice = requests.sort_values(["hour","surge","rain"]).groupby(
    np.arange(len(requests))//3
).apply(lambda g: g.loc[g["walk_m"].idxmin()])

# compare average predicted completion
ai_avg = ai_choice["p_complete_pred"].mean()
base_avg = closest_choice["p_complete_pred"].mean()
lift = (ai_avg - base_avg) / max(base_avg, 1e-9)

hline("Baseline vs AI policy")
print(f"AI avg predicted completion:      {ai_avg:.3f}")
print(f"Baseline (closest-walk) average:  {base_avg:.3f}")
print(f"Relative lift:                    {100*lift:.1f}%")



Baseline vs AI policy
--------------------------------------------------------------------------------
AI avg predicted completion:      0.689
Baseline (closest-walk) average:  0.636
Relative lift:                    8.3%


In [ ]:
def demo_pickup(hour=18, surge=1.25, rain=0):
    candidates = pd.DataFrame({
        "hour":[hour]*3,
        "surge":[surge]*3,
        "rain":[rain]*3,
        "road_quality":[3,3,3],
        "walk_m":[80,210,320],
        "curb_safety":[0.9,0.6,0.4],
        "eta_min":[3.2 + 0.8*rain, 4.8 + 0.8*rain, 3.9 + 0.8*rain],
        "pickup_k":[0,1,2],
        "user_id":[-1,-1,-1],
        "zone_id":[7,7,7]
    })
    ranked = recommend_pickup(candidates)
    print(f"Context → hour={hour}, surge={surge}, rain={rain}")
    display(ranked[["pickup_k","p_complete_pred","walk_m","curb_safety","eta_min","why"]])

hline("What-if: sunny vs rainy")
demo_pickup(hour=18, surge=1.25, rain=0)
demo_pickup(hour=18, surge=1.25, rain=1)



What-if: sunny vs rainy
--------------------------------------------------------------------------------
Context → hour=18, surge=1.25, rain=0


,pickup_k,p_complete_pred,walk_m,curb_safety,eta_min,why
0,0,0.775,80,0.9,3.2,"safer curb, faster ETA, shorter walk"
1,1,0.630,210,0.6,4.8,"safer curb, shorter walk"
2,2,0.601,320,0.4,3.9,faster ETA


Context → hour=18, surge=1.25, rain=1


,pickup_k,p_complete_pred,walk_m,curb_safety,eta_min,why
0,0,0.747,80,0.9,4.0,"safer curb, faster ETA, shorter walk"
2,2,0.559,320,0.4,4.7,faster ETA
1,1,0.553,210,0.6,5.6,"safer curb, shorter walk"


In [ ]:
MAX_WALK_M = 220
CURFEW_STREETS = set()  # could hold street IDs; empty for sim
BLOCKED_ZONES = set()   # e.g., protests/closures; empty for sim

def apply_guardrails(candidates_df, user_is_elder=False, zone_blocked=False):
    df = candidates_df.copy()

    # elderly users: tighter walk constraint
    max_walk = 150 if user_is_elder else MAX_WALK_M
    df = df[df["walk_m"] <= max_walk]

    # blocked zones: (flag only)
    if zone_blocked:
        df = df.iloc[0:0]  # empty

    # If everything filtered out, fallback = closest safe curb under 300m
    if df.empty:
        fallback = candidates_df.sort_values(["walk_m","eta_min"]).iloc[[0]].copy()
        fallback["why"] = "fallback: closest relatively safe curb"
        return fallback

    # Score remaining and return ranked
    ranked = recommend_pickup(df)
    return ranked

hline("Guardrail demo (elderly user → stricter walk rule)")
demo = request_candidates.copy()
ranked_guard = apply_guardrails(demo, user_is_elder=True, zone_blocked=False)
display(ranked_guard[["pickup_k","p_complete_pred","walk_m","curb_safety","eta_min","why"]])



Guardrail demo (elderly user → stricter walk rule)
--------------------------------------------------------------------------------


,pickup_k,p_complete_pred,walk_m,curb_safety,eta_min,why
0,0,0.775,80,0.9,3.2,"safer curb, faster ETA, shorter walk"


In [ ]:
# rich features + HistGradientBoosting + quick tuning + calibration ===
# improve ranking quality (AUC) versus the baseline 0.60

import numpy as np, pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss
from sklearn.calibration import CalibratedClassifierCV
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

hline("Rich feature pipeline + HistGB")

# features from existing 'trips' table
# augment the baseline features with:
# - Cyclic time encoding (sin/cos hour)
# - Log transforms (walk_m, eta_min)
# - Interaction features (rain*walk_m, rain*eta_min, surge*walk_m, curb*road_quality)
# - Quantile bins for walk/eta to let trees capture bucketing effects easily

BASE_FEATS = ["hour","surge","rain","road_quality","walk_m","curb_safety","eta_min"]
y_all = trips["completed"].astype(int).values

df = trips[["hour","surge","rain","road_quality","walk_m","curb_safety","eta_min"]].copy()

df["hour_sin"] = np.sin(2*np.pi*df["hour"]/24)
df["hour_cos"] = np.cos(2*np.pi*df["hour"]/24)

df["log_walk"] = np.log1p(df["walk_m"])
df["log_eta"]  = np.log1p(df["eta_min"])

df["rain_x_walk"]  = df["rain"] * df["walk_m"]
df["rain_x_eta"]   = df["rain"] * df["eta_min"]
df["surge_x_walk"] = df["surge"] * df["walk_m"]
df["curb_x_road"]  = df["curb_safety"] * (5 - df["road_quality"])  # high when curb is low & road is poor

# buckets for walk/eta to model thresholds like
binner = KBinsDiscretizer(n_bins=6, encode="ordinal", strategy="quantile")
df[["walk_bin","eta_bin"]] = binner.fit_transform(df[["walk_m","eta_min"]])

RICH_FEATS = list(df.columns)
X_all = df.values

# train/valid/test split (as before)
X_tr, X_te, y_tr, y_te = train_test_split(X_all, y_all, test_size=0.25, random_state=42, stratify=y_all)

# model: HistGradientBoosting
hgb = HistGradientBoostingClassifier(
    learning_rate=0.06,
    max_depth=6,
    max_iter=300,
    l2_regularization=0.0,
    early_stopping=True,
    validation_fraction=0.1,
    random_state=42
)

#  randomized search around impactful knobs
param_dist = {
    "learning_rate": [0.03, 0.05, 0.06, 0.08, 0.1],
    "max_depth": [3, 4, 5, 6, 7],
    "max_iter": [150, 220, 300, 400],
    "l2_regularization": [0.0, 1e-4, 1e-3, 1e-2],
    "min_samples_leaf": [20, 50, 100]
}
search = RandomizedSearchCV(
    estimator=hgb,
    param_distributions=param_dist,
    n_iter=20,
    scoring="roc_auc",
    cv=3,
    random_state=42,
    n_jobs=-1,
    verbose=0
)

search.fit(X_tr, y_tr)
model_hgb = search.best_estimator_

# evaluate vs. original model

try:
    X_baseline_te = trips.loc[X_te.shape[0]*-1:].copy()
except Exception:
    pass

#  scores for new model
p_hgb = model_hgb.predict_proba(X_te)[:,1]
auc_hgb = roc_auc_score(y_te, p_hgb)
pr_hgb  = average_precision_score(y_te, p_hgb)
brier_hgb = brier_score_loss(y_te, p_hgb)

#report side-by-side
baseline_ok = False
try:
    p_base = pickup_model.predict_proba(Xte)[:,1]
    auc_base = roc_auc_score(yte, p_base)
    pr_base  = average_precision_score(yte, p_base)
    brier_base = brier_score_loss(yte, p_base)
    baseline_ok = True
except Exception:
    pass

hline("Test metrics (HistGB rich features)")
print(f"HistGB  AUC:   {auc_hgb:.3f}")
print(f"HistGB  PR-AUC:{pr_hgb:.3f}")
print(f"HistGB  Brier: {brier_hgb:.3f}")

if baseline_ok:
    print("-"*80)
    print(f"Baseline AUC:   {auc_base:.3f}")
    print(f"Baseline PR-AUC:{pr_base:.3f}")
    print(f"Baseline Brier: {brier_base:.3f}")

# calibrate (fast, prefit) for better probability quality

Xfit, Xcal, yfit, ycal = train_test_split(X_tr, y_tr, test_size=0.2, random_state=42, stratify=y_tr)

model_hgb.fit(Xfit, yfit)

cal_iso = CalibratedClassifierCV(model_hgb, method="isotonic", cv="prefit")
cal_iso.fit(Xcal, ycal)
p_cal = cal_iso.predict_proba(X_te)[:,1]

hline("Calibration (isotonic, prefit)")
print(f"AUC   (uncal → cal): {auc_hgb:.3f} → {roc_auc_score(y_te, p_cal):.3f}")
print(f"Brier (uncal → cal): {brier_hgb:.3f} → {brier_score_loss(y_te, p_cal):.3f}")



RICH_ORDER = RICH_FEATS

def _featurize_candidates_rich(cand: pd.DataFrame) -> pd.DataFrame:
    out = cand.copy()
    out["hour_sin"] = np.sin(2*np.pi*out["hour"]/24)
    out["hour_cos"] = np.cos(2*np.pi*out["hour"]/24)
    out["log_walk"] = np.log1p(out["walk_m"])
    out["log_eta"]  = np.log1p(out["eta_min"])
    out["rain_x_walk"]  = out["rain"] * out["walk_m"]
    out["rain_x_eta"]   = out["rain"] * out["eta_min"]
    out["surge_x_walk"] = out["surge"] * out["walk_m"]
    out["curb_x_road"]  = out["curb_safety"] * (5 - out["road_quality"])
    tmp = cand[["walk_m","eta_min"]].copy()
    kb = KBinsDiscretizer(n_bins=6, encode="ordinal", strategy="quantile")
    kb.fit(trips[["walk_m","eta_min"]])  # fit on global dist from training sim
    out[["walk_bin","eta_bin"]] = kb.transform(tmp)
    return out[RICH_ORDER]

def recommend_pickup_rich(candidates_df: pd.DataFrame, model=cal_iso):
    feats = _featurize_candidates_rich(candidates_df)
    proba = model.predict_proba(feats)[:,1]
    out = candidates_df.copy()
    out["p_complete_pred"] = proba
    #  “why” based on richer signals
    def why(row, med=candidates_df.median(numeric_only=True)):
        reasons = []
        if row["curb_safety"] >= med["curb_safety"]: reasons.append("safer curb")
        if row["eta_min"] <= med["eta_min"]: reasons.append("faster ETA")
        if row["walk_m"] <= med["walk_m"]: reasons.append("shorter walk")
        if row["rain"]==1 and row["eta_min"] <= med["eta_min"]+0.5: reasons.append("rain-robust")
        return ", ".join(reasons) if reasons else "balanced tradeoffs"
    out["why"] = out.apply(why, axis=1)
    return out.sort_values("p_complete_pred", ascending=False)

req = pd.DataFrame({
    "hour":[18,18,18],
    "surge":[1.25,1.25,1.25],
    "rain":[0,0,0],
    "road_quality":[3,3,3],
    "walk_m":[80,210,320],
    "curb_safety":[0.9,0.6,0.4],
    "eta_min":[3.2,4.8,3.9],
    "pickup_k":[0,1,2],
    "user_id":[-1,-1,-1],
    "zone_id":[7,7,7]
})

hline("OLD model vs NEW model")
old_rank = recommend_pickup(req, model=pickup_model)[["pickup_k","p_complete_pred"]].head(1)
new_rank = recommend_pickup_rich(req, model=cal_iso)[["pickup_k","p_complete_pred"]].head(1)
print("Baseline top:", old_rank.to_dict(orient="records")[0])
print("New model top:", new_rank.to_dict(orient="records")[0])

hline("NEW model ranking")
display(recommend_pickup_rich(req, model=cal_iso)[["pickup_k","p_complete_pred","walk_m","curb_safety","eta_min","why"]])



Rich feature pipeline + HistGB
--------------------------------------------------------------------------------

Test metrics (HistGB rich features)
--------------------------------------------------------------------------------
HistGB  AUC:   0.605
HistGB  PR-AUC:0.704
HistGB  Brier: 0.229
--------------------------------------------------------------------------------
Baseline AUC:   0.604
Baseline PR-AUC:0.704
Baseline Brier: 0.229

Calibration (isotonic, prefit)
--------------------------------------------------------------------------------
AUC   (uncal → cal): 0.605 → 0.603
Brier (uncal → cal): 0.229 → 0.229

OLD model vs NEW model
--------------------------------------------------------------------------------
Baseline top: {'pickup_k': 0, 'p_complete_pred': 0.7622225086473581}
New model top: {'pickup_k': 0, 'p_complete_pred': 0.7645011600928074}

NEW model ranking
--------------------------------------------------------------------------------


/usr/local/lib/python3.12/dist-packages/sklearn/calibration.py:333: UserWarning: The `cv='prefit'` option is deprecated in 1.6 and will be removed in 1.8. You can use CalibratedClassifierCV(FrozenEstimator(estimator)) instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but HistGradientBoostingClassifier was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but HistGradientBoostingClassifier was fitted without feature names
  warnings.warn(


,pickup_k,p_complete_pred,walk_m,curb_safety,eta_min,why
0,0,0.765,80,0.9,3.2,"safer curb, faster ETA, shorter walk"
1,1,0.664,210,0.6,4.8,"safer curb, shorter walk"
2,2,0.632,320,0.4,3.9,faster ETA
